In [ ]:
def cp2_recursion(d):
    if d <= 0:
        return 0

    if d == 1:
        return 1  # N_d = 1

    acc = 0
    for k in range(1, d):  # k from 1 to d - 1
        l = d - k
        acc += cp2_recursion(k) * cp2_recursion(l) * \
            (k ^ 2*l ^ 2*binomial(3*d - 4, 3*k - 2) -
                k ^ 3*l*binomial(3*d - 4, 3*k - 1))
    return acc


In [ ]:
def bipart(l):
    if len(l) == 0:
        return [[[], []]]
    elif len(l) == 1:
        k = l[0]
        return [[[], [k]], [[k], []]]
    else:
        ll = [l[i] for i in range(len(l)-1)]
        R = bipart(ll)
        R1 = [[R[i][0] + [l[-1]], R[i][1]] for i in range(len(R))]
        R2 = [[R[i][0], R[i][1] + [l[-1]]] for i in range(len(R))]
        return R1 + R2


In [ ]:
# our own version of code
# input:
#   n - dimension of complex projective space
#   d - degree of rational curve
#   l - list of cohomology classes represented by integers
# output: gromov-witten invariant of complex projective n space
def gw_invariant(n, d, l):
    l.sort()    # asceding order
    l.reverse() # to descending order
    k = len(l)

    # cannot not have cohomology class of degree
    # greater than n
    for c_d in l:
        if c_d > n:
            return 0

    # dimension condition at (7.5.4)
    if sum(l) != n*(d+1) + d + k - 3:
        return 0

    # exercise 7.5.3 and dimension condition
    if k == 0 or k == 1:
        return 0

    # proposition 7.5.6 zero axiom
    if d == 0:
        if k == 3:  #and sum(l) == n: #implied from dimension condition
            return 1
        else:
            return 0

    if k == 2:
        # exercise 7.5.3
        if d == 1 and l == [n, n]:
            return 1
        else:
            return 0  # TODO

    # l[n-1] = l[l.length - 1]
    if l[-1] == 0:
        return 0

    # divisor axiom
    elif l[-1] == 1:
        l.pop(-1)
        return d*gw_invariant(n, d, l)

    # CP 2 situation
    elif n == 2:
        if l == [2]*(3*d - 1):
            return cp2_recursion(d)
        else:
            return 0

    else: #notation are from https://www.ihes.fr/~maxim/TEXTS/WithManinCohFT.pdf

        li = l[0] # gamma_1
        lj = l[1] # gamma_2
        lk = l[-1] - 1 # delta, where l[k-1] = delta + delta'
        ll = 1         # delta', degree 2 class

        res = gw_invariant(n, d, [l[i] for i in range(2, k-1)] + \
            [ll, lj, li+lk])


        # bipart gamma_3,... to gamma_{k-1}]
        S = bipart([l[i] for i in range(2, k-1)]) 
        # k = len(l)
        # S = [(s1, s2), (s1', s2'),....    ]

        for s in S:
            S1 = s[0]
            S2 = s[1]
            
            k1 = len(S1)

            for d1 in range(1, d+1):
                d2 = d - d1

                # dimension condition
                a_rhs = (d1 + 1)*n + d1 + k1 - sum(S1) - li - lk
                b_rhs = n - a_rhs
                if a_rhs >= 0 and a_rhs <= n:
                    rhs1 = gw_invariant(n, d1, S1 + [li, lk, a_rhs])

                    # apply divisor axiom
                    rhs2 = gw_invariant(n, d2, S2 + [lj, ll, b_rhs])

                    res = res + rhs1*rhs2

                a_lhs = (d1 + 1)*n + d1 + k1 - sum(S1) - li - lj
                b_lhs = n - a_lhs
                if a_lhs >= 0 and a_lhs <= n and len(S2) > 0:
                    
                    # the original GW invariant we want is this case 
                    # when len(S2) = 0
                    lhs1 = gw_invariant(n, d1, S1 + [li, lj, a_lhs])
                    
                    # apply divisor axiom
                    lhs2 = gw_invariant(n, d2, S2 + [lk, ll, b_lhs])

                    res  = res - lhs1*lhs2

        return res
